## Electricity price prediction for NYC

In [1]:
import pandas as pd

import numpy as np
from sklearn import preprocessing
from sklearn import metrics
import matplotlib.pylab as plt
import datetime as dt
import time

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Convolution1D, MaxPooling1D, AtrousConvolution1D, RepeatVector
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.layers.wrappers import Bidirectional
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import *
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.initializers import *

Using TensorFlow backend.


In [2]:
df_data = pd.read_csv('full_diff_data_2016_2017.csv', parse_dates=['time_stamp', 'time_stamp_local'])
df_data.set_index('time_stamp', inplace = True)

In [3]:
df_data.shape

(17520, 10)

In [4]:
df_data.columns

Index(['time_stamp_local', 'price', 'price_24hr_ago', 'price_diff', 'load',
       'load_24hr_ago', 'load_diff', 'temp', 'temp_24hr_ago', 'temp_diff'],
      dtype='object')

In [5]:
def train_test_ts_split(X, y, percentage=0.9):
    X_train = X[0:int(len(X) * percentage)]
    Y_train = y[0:int(len(y) * percentage)]
    
    #X_train, Y_train = shuffle_Xy(X_train, Y_train)

    X_test = X[int(len(X) * percentage):]
    Y_test = y[int(len(X) * percentage):]

    return X_train, X_test, Y_train, Y_test

In [6]:
scaler_price = preprocessing.StandardScaler().fit(df_data.price.values.reshape(-1, 1))
scaler_load = preprocessing.StandardScaler().fit(df_data.load.values.reshape(-1, 1))
scaler_temp = preprocessing.StandardScaler().fit(df_data.temp.values.reshape(-1, 1))
# # scaler_price = preprocessing.MinMaxScaler().fit(df_data.price_diff.values.reshape(-1, 1))
# # scaler_load = preprocessing.MinMaxScaler().fit(df_data.load_diff.values.reshape(-1, 1))
# # scaler_temp = preprocessing.MinMaxScaler().fit(df_data.temp_diff.values.reshape(-1, 1))

df_data.loc[:,'price_norm'] = scaler_price.transform(df_data.price.values.reshape(-1, 1))
df_data.loc[:,'load_norm'] = scaler_load.transform(df_data.load.values.reshape(-1, 1))
df_data.loc[:,'temp_norm'] = scaler_temp.transform(df_data.temp.values.reshape(-1, 1))

In [7]:
WINDOW = 24 * 7 - 1
EMB_SIZE = 3
STEP = 1
FORECAST = 1

X, Y = [], []
for i in range(0, df_data.shape[0], STEP): 
    try:
        price_byweek = df_data.price_norm.values[i: i + WINDOW]
        load_byweek = df_data.load_norm.values[i: i + WINDOW]
        temp_byweek = df_data.temp_norm.values[i: i + WINDOW]

#         price_byweek = df_data.price.values[i: i + WINDOW]
#         load_byweek = df_data.load.values[i: i + WINDOW]
#         temp_byweek = df_data.temp.values[i: i + WINDOW]

        
#         price_norm = (np.array(price_byweek) - np.mean(price_byweek)) / np.std(price_byweek)
#         load_norm = (np.array(load_byweek) - np.mean(load_byweek)) / np.std(load_byweek)
#         temp_norm = (np.array(temp_byweek) - np.mean(temp_byweek)) / np.std(temp_byweek)
        
#         x_i = np.column_stack((price_norm, load_norm, temp_norm))
        x_i = np.column_stack((price_byweek, load_byweek, temp_byweek))

        y_i = df_data.price.values[i + WINDOW + FORECAST]
    except Exception as e:
        break

    X.append(x_i)
    Y.append(y_i)

X, Y = np.array(X), np.array(Y)

In [8]:
X.shape, Y.shape

((17352, 167, 3), (17352,))

In [9]:
X_train, X_test, Y_train, Y_test = train_test_ts_split(X, Y, percentage = 0.9)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((15616, 167, 3), (1736, 167, 3), (15616,), (1736,))

In [10]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], EMB_SIZE))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], EMB_SIZE))
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((15616, 167, 3), (1736, 167, 3), (15616,), (1736,))

In [ ]:
model = Sequential()
model.add(Convolution1D(input_shape = (WINDOW, EMB_SIZE),
                        nb_filter=32,
                        filter_length=24,
                        border_mode='same'))

model.add(BatchNormalization())
model.add(Activation('relu'))
# model.add(Dropout(0.5))


model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid'))
model.add(Convolution1D(nb_filter=16,
                        filter_length=2,
                        border_mode='same'))

model.add(BatchNormalization())
model.add(Activation('relu'))


model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid'))
model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
# model.add(Dropout(0.3))

model.add(Dense(1))
model.add(Activation('linear'))

model.compile(optimizer='adam', 
              loss='mse')
history = model.fit(X_train, Y_train, 
          nb_epoch = 30, 
          batch_size = 128, 
          verbose=1, 
          validation_split=0.2,
          shuffle=True)

/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(kernel_size=24, input_shape=(167, 3), filters=32, padding="same")`
  """
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(kernel_size=2, padding="same", filters=16)`
  from ipykernel import kernelapp as app
/anaconda/envs/py35/lib/python3.5/site-packages/keras/models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 12492 samples, validate on 3124 samples
Epoch 1/30
12492/12492 [==============================] - 11s 909us/step - loss: 813.2901 - val_loss: 473.8110
Epoch 2/30
12492/12492 [==============================] - 11s 854us/step - loss: 663.7734 - val_loss: 383.4331
Epoch 3/30
12492/12492 [==============================] - 11s 848us/step - loss: 630.6682 - val_loss: 326.8854
Epoch 4/30
12492/12492 [==============================] - 12s 964us/step - loss: 604.3726 - val_loss: 310.5981
Epoch 5/30
12492/12492 [==============================] - 11s 858us/step - loss: 586.7111 - val_loss: 297.2561
Epoch 6/30
12492/12492 [==============================] - 10s 830us/step - loss: 576.5124 - val_loss: 289.1455
Epoch 7/30
12492/12492 [==============================] - 11s 858us/step - loss: 558.4107 - val_loss: 287.3028
Epoch 8/30
12492/12492 [==============================] - 12s 921us/step - loss: 546.2055 - val_loss: 295.6822
Epoch 9/30
12492/12492 [==============================] - 11s 8

In [ ]:
print(model.summary())

In [ ]:
model.save('model.h5')
#model.load_weights("model.hdf5")
pred = model.predict(np.array(X_test))

In [ ]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

plt.figure()
# plt.plot(scaler_price.inverse_transform(Y_test))
# plt.plot(scaler_price.inverse_transform(pred))
plt.plot(Y_test)
plt.plot(pred)


plt.title('time series')
# plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Y_test', 'pred'], loc='best')
plt.xlim(6000, 6100)
plt.ylim(0,100)
plt.show()

In [ ]:
# np.sqrt(metrics.mean_squared_error(scaler_price.inverse_transform(Y_test), scaler_price.inverse_transform(pred)))
np.sqrt(metrics.mean_squared_error(Y_test.ravel(), pred.ravel()))

In [ ]:
pred[0]

In [ ]:
# metrics.mean_absolute_error(scaler_price.inverse_transform(Y_test), scaler_price.inverse_transform(pred))
np.sqrt(metrics.mean_absolute_error(Y_test.ravel(), pred.ravel()))